In [1]:
import pandas as pd
from pathlib import Path
from enum import Enum
from dataclasses import dataclass
from typing import Optional
from sklearn.linear_model import LinearRegression

In [36]:
class AbaloneDataColumnsTypes(Enum):
    SEX = 'Sex' 
    LEN = 'Length'
    DIAM = 'Diameter'
    HEIGHT =  'Height'
    W_WEIGHT =  'Whole weight'
    SHUCK_WEIGHT = 'Shucked weight'
    V_WEIGHT = 'Viscera weight'
    SHELL_WEIGHT = 'Shell weight'
    RINGS = 'Rings'

    def __str__(self) -> str:
        return self.value


@dataclass
class CommonParams:
    data_path = Path('./data')

    num_iterations = 1000
    selection_ratio = 0.1
    exclude_columns = [AbaloneDataColumnsTypes.SEX]
    predict_column = AbaloneDataColumnsTypes.RINGS

In [37]:
def get_abalone_dataset(path: Path, abaloneDataColumns) -> pd.DataFrame:
    dataset_features_path = path / 'abalone' / 'abalone.data'

    dataset = pd.read_csv(dataset_features_path, header = None)
    dataset.columns = abaloneDataColumns
    return dataset

In [38]:
def preprocess_abalone_dataset(dataset: pd.DataFrame, exclude_columns: Optional[AbaloneDataColumnsTypes] = None
                               ) -> pd.DataFrame:
    if exclude_columns:
        for column in exclude_columns:
            dataset = dataset.drop(columns=column.value)
    return dataset

In [39]:
def get_random_subset(dataset: pd.DataFrame, selection_ratio: float) -> pd.DataFrame:
    dataset_size = len(dataset)
    subset_size = int(dataset_size * selection_ratio)
    subset = dataset.sample(subset_size)
    return subset

In [40]:
def x_y_split(dataset: pd.DataFrame, predicted_column: AbaloneDataColumnsTypes) -> tuple[pd.DataFrame, pd.DataFrame]:
    data_x = dataset.copy()
    data_x = data_x.drop(columns=[predicted_column.value])

    data_y = dataset[predicted_column.value]
    return data_x, data_y

In [41]:
def scaleMinMax(frame: pd.DataFrame) -> pd.DataFrame:
    return (frame - frame.min()) / (frame.max() - frame.min())

In [48]:
def selectValuable(dataset: pd.DataFrame,
                   quantiles: tuple[float, float] = (0.5, 0.95),
                   max_delta: float = 0.25) -> pd.DataFrame:
    scaled_dataset = scaleMinMax(dataset)
    delta = abs(scaled_dataset.quantile(quantiles[0]) - scaled_dataset.quantile(quantiles[1]))
    absolute_delta = abs(delta)
    selected_parameters = absolute_delta.loc[absolute_delta < max_delta]
    return selected_parameters

In [53]:
params = CommonParams()

abaloneDataColumns = [str(columnType) for columnType in AbaloneDataColumnsTypes]

abaloneProcessedDataColumns = [str(columnType) for columnType in AbaloneDataColumnsTypes
                               if columnType is not params.predict_column and columnType not in params.exclude_columns]

dataset = get_abalone_dataset(params.data_path, abaloneDataColumns)
processed_dataset = preprocess_abalone_dataset(dataset, params.exclude_columns)

coef_history = []
for iteration in range(params.num_iterations):
    subset = get_random_subset(processed_dataset, params.selection_ratio)
    
    data_x, data_y = x_y_split(subset, params.predict_column)
    regression = LinearRegression()
    regression.fit(data_x, data_y)
    coef_history.append(regression.coef_)

frameHistory = pd.DataFrame(coef_history, columns=abaloneProcessedDataColumns)
important_parameters = selectValuable(frameHistory).sort_values()
important_parameters

Viscera weight    0.203565
Shucked weight    0.227526
Length            0.228401
Shell weight      0.243445
dtype: float64